In [1]:
import spotipy 
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm 
import pandas as pd 

In [2]:
client_id = '12824720980943c684531a5514a21114'
client_secret = '6eb9eb97a614492393a1cfe2e606b227'

In [3]:
artist_name = []
track_name = []
popularity = []
track_id = []

In [6]:
scope = 'user-top-read'
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret= client_secret,
        scope=scope,
        redirect_uri='https://localhost/'
    )
)

ranges = ['short_term', 'medium_term', 'long_term']

for sp_range in ranges:
    print("range:",sp_range)
    results = sp.current_user_top_tracks(time_range=sp_range, limit=50)
    for i, item in enumerate(results['items']):
        artist_name.append(item['artists'][0]['name'])
        track_name.append(item['name'])
        track_id.append(item['id'])
        popularity.append(item['popularity'])

range: short_term


KeyboardInterrupt: Interrupted by user

In [7]:
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021']
for year in years:
    for i in tqdm(range(0, 1000, 50)):
        track_results = sp.search(q=f'year:{year}', type='track', limit=50, offset=i)
        for i, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])

  0%|          | 0/20 [00:26<?, ?it/s]


KeyboardInterrupt: Interrupted by user

In [9]:
top_artists_name = []
top_artist_uri = []
for sp_range in ['short_term', 'medium_term', 'long_term']:
    print("range:", sp_range)

    results = sp.current_user_top_artists(time_range=sp_range, limit=50)

    for item in results['items']:
        top_artists_name.append(item['name'])
        top_artist_uri.append(item['uri'])

print(top_artists_name)
print(top_artist_uri)

range: short_term


KeyboardInterrupt: Interrupted by user

In [8]:
for artist, uri in tqdm(zip(top_artists_name, top_artist_uri), total=len(top_artist_uri)):
    response = sp.artist_top_tracks(uri, country='NP')
    for track in response['tracks']:
        artist_name.append(artist)
        track_name.append(track['name'])
        track_id.append(track['id'])
        popularity.append(track['popularity'])

NameError: name 'top_artists_name' is not defined

In [10]:
df_tracks = pd.DataFrame(
    {
        'artist_name': artist_name,
        'track_name':track_name,
        'track_id':track_id,
        'popularity':popularity
    }
)
print(df_tracks.shape)
df_tracks

(0, 4)


,artist_name,track_name,track_id,popularity


In [11]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   artist_name  0 non-null      float64
 1   track_name   0 non-null      float64
 2   track_id     0 non-null      float64
 3   popularity   0 non-null      float64
dtypes: float64(4)
memory usage: 124.0 bytes


In [12]:
grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

0

In [13]:
df_tracks.shape 

(0, 4)

In [14]:
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i: i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:', None_counter)

Number of tracks where no audio features were available: 0


In [15]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 0


In [16]:
audio_features_df = pd.DataFrame.from_dict(rows, orient='columns')
print("Shape of the dataset:", audio_features_df.shape)
audio_features_df

Shape of the dataset: (0, 0)


""


In [17]:
audio_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame

In [18]:
columns_to_drop = ['analysis_url', 'track_href', 'type', 'uri']
audio_features_df.drop(columns_to_drop, axis=1,inplace=True)

audio_features_df.rename(columns={'id': 'track_id'}, inplace=True)

audio_features_df.shape

KeyError: "['analysis_url', 'track_href', 'type', 'uri'] not found in axis"

In [19]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks, audio_features_df, on='track_id', how='inner')
df

KeyError: 'track_id'

In [20]:
df.info()

NameError: name 'df' is not defined

In [21]:
df[df.duplicated(subset=['artist_name', 'track_name'], keep=False)]

NameError: name 'df' is not defined

In [22]:
df.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
df[df.duplicated(subset=['artist_name', 'track_name'], keep=False)]

NameError: name 'df' is not defined

In [23]:
df.shape

NameError: name 'df' is not defined

In [24]:
df.to_csv('./data/spotify_audio_features.csv', index=False)

NameError: name 'df' is not defined